In [225]:
import sklearn
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap

In [140]:
PENGUIN_DATASET = '/Users/philipp/Documents/02_Master_Uni/Uni_Tübingen/Semester_1/06 Data Literacy/02 Project/penguins_final_with_era5.csv'
penguin_df = pd.read_csv(PENGUIN_DATASET)

penguin_df.columns

Index(['Unnamed: 0', 'track_id', 'date_gmt', 'latitude_mean', 'longitude_mean',
       'lat_colony_mean', 'lon_colony_mean', 'km_to_colony_mean',
       'km_since_last_measure_mean', 'delta_km_north_mean',
       'delta_km_south_mean', 'delta_km_east_mean', 'delta_km_west_mean',
       'minutes_since_last_measure_mean', 'latitude_std', 'longitude_std',
       'lat_colony_std', 'lon_colony_std', 'km_to_colony_std',
       'km_since_last_measure_std', 'delta_km_north_std', 'delta_km_south_std',
       'delta_km_east_std', 'delta_km_west_std',
       'minutes_since_last_measure_std', 'latitude_min', 'longitude_min',
       'lat_colony_min', 'lon_colony_min', 'km_to_colony_min',
       'km_since_last_measure_min', 'delta_km_north_min', 'delta_km_south_min',
       'delta_km_east_min', 'delta_km_west_min',
       'minutes_since_last_measure_min', 'latitude_max', 'longitude_max',
       'lat_colony_max', 'lon_colony_max', 'km_to_colony_max',
       'km_since_last_measure_max', 'delta_km_nort

In [28]:
# define used colonies:
species = penguin_df['common_name'].unique()

available_years = [str(x) for x in range(1996, 2018)]
available_years = np.array(available_years)
penguin_df['date_gmt'] = pd.to_datetime(penguin_df['date_gmt'])
penguin_df['year'] = penguin_df['date_gmt'].dt.year.astype(str)
sufficient_pairs = []

for specie in species:
    colonies_with_data = []
    for colony in penguin_df[penguin_df['common_name'] == specie]['colony_name'].unique():
        years_with_data = penguin_df[(penguin_df['common_name'] == specie) & (penguin_df['colony_name'] == colony)]['year'].unique()
        if len(years_with_data) >= 8:
            colonies_with_data.append(colony)
    if colonies_with_data:
        sufficient_pairs.append({'specie': specie, 'colonies': colonies_with_data})

print(sufficient_pairs)

[{'specie': 'Adelie Penguin', 'colonies': ['Admiralty Bay']}, {'specie': 'Chinstrap Penguin', 'colonies': ['Admiralty Bay', 'Cape Shirreff']}, {'specie': 'Gentoo Penguin', 'colonies': ['Admiralty Bay', 'Cape Shirreff']}]


In [5]:
# filter df for adelie penguins and king george island
df = penguin_df[(penguin_df['common_name'] == 'Adelie Penguin') & (penguin_df['colony_name'] == 'King George Island')]
#df = penguin_df[(penguin_df['common_name'] == 'Adelie Penguin')]

In [224]:
def prep(df):
    df = df.copy()
    df['date_gmt'] = pd.to_datetime(df['date_gmt'])
    df = df.sort_values(by='date_gmt')

    # Feature Engineering
    df['day_of_year'] = df['date_gmt'].dt.dayofyear
    df['week_of_year'] = df['date_gmt'].dt.isocalendar().week
    df['month'] = df['date_gmt'].dt.month
    df['year'] = df['date_gmt'].dt.year

    # Create lag features for 'tp' and 't2m'
    for lag in range(1, 4):
        df[f'tp_lag_{lag}'] = df['tp'].shift(lag)
        df[f't2m_lag_{lag}'] = df['t2m'].shift(lag)

    # Cyclical features for temporal variables
    df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.0)
    df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.0)
    df['week_of_year_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52.0)
    df['week_of_year_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52.0)

    # Add lag features for additional variables
    for lag in range(1, 4):
        df[f'sst_lag_{lag}'] = df['sst'].shift(lag)
        df[f'siconc_lag_{lag}'] = df['siconc'].shift(lag)
        df[f'sd_lag_{lag}'] = df['sd'].shift(lag)
        df[f'rsn_lag_{lag}'] = df['rsn'].shift(lag)
        df[f'avg_smr_lag_{lag}'] = df['avg_smr'].shift(lag)

    # # Rolling averages and cumulative sums for additional variables
    # df['sst_rolling_7'] = df['sst'].rolling(window=7, min_periods=1).mean()
    # df['siconc_rolling_7'] = df['siconc'].rolling(window=7, min_periods=1).mean()
    # df['sd_cumsum'] = df['sd'].cumsum()
    # df['rsn_cumsum'] = df['rsn'].cumsum()

    # Drop rows with missing values
    df = df.dropna()

    # Define feature set and target variable
    features = [
        'tp', 't2m',  'sst', 'siconc', 
        'tp_lag_1', 'tp_lag_2', 'tp_lag_3',
        't2m_lag_1', 't2m_lag_2', 't2m_lag_3', 
        'sst_lag_1', 'sst_lag_2', 'sst_lag_3', 
         'siconc_lag_1', 'siconc_lag_2', 'siconc_lag_3', 
        'year',
        #  'week_of_year_sin', 'week_of_year_cos', 
        'day_of_year_sin', 'day_of_year_cos',
        #   'sst_rolling_7', 'siconc_rolling_7', 'sd_cumsum', 'rsn_cumsum'
    ]
    target = 'km_to_colony_mean'
    # target = 'km_since_last_measure_mean'

    X = df[features]
    y = df[target]

    # X100 = shap.utils.sample(X, 100) # 100 samples for use in Shapley value calculations

    # Standardize features
    scaler = StandardScaler()
    scaled_features = ['sst', 'siconc', 'tp', 't2m']
    #X[scaled_features] = scaler.fit_transform(X[scaled_features])
    #X_scaled = scaler.fit_transform(X)

    return X, y, scaler, features, scaled_features


In [143]:
def use_ridge_lasso(X, y):

    # Ridge and Lasso models
    ridge_model = Ridge(alpha=1.0)
    lasso_model = Lasso(alpha=0.1)

    # TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)

    # Lists to store evaluation metrics
    ridge_r2_scores = []
    ridge_rmse_scores = []
    lasso_r2_scores = []
    lasso_rmse_scores = []

    # Model training and evaluation
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Ridge Regression
        ridge_model.fit(X_train, y_train)
        ridge_y_pred = ridge_model.predict(X_test)
        ridge_r2_scores.append(r2_score(y_test, ridge_y_pred))
        ridge_rmse_scores.append(np.sqrt(mean_squared_error(y_test, ridge_y_pred)))

        # Lasso Regression
        lasso_model.fit(X_train, y_train)
        lasso_y_pred = lasso_model.predict(X_test)
        lasso_r2_scores.append(r2_score(y_test, lasso_y_pred))
        lasso_rmse_scores.append(np.sqrt(mean_squared_error(y_test, lasso_y_pred)))

    # Print results
    print(f"Ridge Average R^2 Score: {np.mean(ridge_r2_scores):.3f}")
    print(f"Ridge Average RMSE: {np.mean(ridge_rmse_scores):.3f}")
    print(f"Lasso Average R^2 Score: {np.mean(lasso_r2_scores):.3f}")
    print(f"Lasso Average RMSE: {np.mean(lasso_rmse_scores):.3f}")

    # Save and print model coefficients
    coefficients = pd.DataFrame({
        'Feature': X.columns,
        'Ridge Coef': ridge_model.coef_,
        'Lasso Coef': lasso_model.coef_
    })
    #print(coefficients)

    coefficients['Ridge Importance'] = coefficients['Ridge Coef'].abs()
    coefficients['Lasso Importance'] = coefficients['Lasso Coef'].abs()

    # Sort by importance (descending order)
    ridge_sorted = coefficients.sort_values(by='Ridge Importance', ascending=False)
    lasso_sorted = coefficients.sort_values(by='Lasso Importance', ascending=False)

    # Print sorted coefficients
    #print("Ridge Coefficients Sorted by Importance:")
    #print(ridge_sorted[['Feature', 'Ridge Coef', 'Ridge Importance']])

    print("\nLasso Coefficients Sorted by Importance:")
    print(lasso_sorted[['Feature', 'Lasso Coef', 'Lasso Importance']])

In [125]:
def use_random_forest(X, y, features):
    rf_model = RandomForestRegressor(
        n_estimators=200,  # Number of trees
        max_depth=20,      # Maximum depth of trees
        random_state=42    # Reproducibility
    )

    tscv = TimeSeriesSplit(n_splits=5)
    # To store evaluation metrics
    rf_r2_scores = []
    rf_rmse_scores = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train Random Forest
        rf_model.fit(X_train, y_train)
        rf_y_pred = rf_model.predict(X_test)
        
        # Evaluate Random Forest
        rf_r2_scores.append(r2_score(y_test, rf_y_pred))
        rf_rmse_scores.append(np.sqrt(mean_squared_error(y_test, rf_y_pred)))

    # Print Random Forest results
    print(f"Random Forest Average R^2 Score: {np.mean(rf_r2_scores):.3f}")
    print(f"Random Forest Average RMSE: {np.mean(rf_rmse_scores):.3f}")

    # Feature importance
    feature_importance = pd.DataFrame({
        'Feature': features,
        'Importance': rf_model.feature_importances_,
    }).sort_values(by='Importance', ascending=False)
    print(feature_importance)


In [126]:
def use_xgb(X, y):    
    xgb_model = XGBRegressor(
        n_estimators=100,  # Number of trees
        max_depth=10,       # Maximum depth of each tree
        learning_rate=0.05, # Learning rate
        random_state=42    # Seed for reproducibility
    )

    # To store evaluation metrics
    xgb_r2_scores = []
    xgb_rmse_scores = []

    # TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train XGBoost model
        xgb_model.fit(X_train, y_train)
        
        # Predictions
        y_pred = xgb_model.predict(X_test)
        
        # Evaluate model
        xgb_r2_scores.append(r2_score(y_test, y_pred))
        xgb_rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    # Print XGBoost results
    print(f"XGBoost Average R^2 Score: {np.mean(xgb_r2_scores):.3f}")
    print(f"XGBoost Average RMSE: {np.mean(xgb_rmse_scores):.3f}")

    xgb_feature_importance = xgb_model.feature_importances_

    # Create a DataFrame for better visualization
    feature_importance_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': xgb_feature_importance
    })

    # Sort by importance
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Print the feature importance
    print(feature_importance_df)

# penguins in general

In [207]:
df = penguin_df
prep(df)
X, y, scaler, features, scaled_features = prep(df)
use_ridge_lasso(X, y)

Ridge Average R^2 Score: 0.021
Ridge Average RMSE: 369.674
Lasso Average R^2 Score: 0.030
Lasso Average RMSE: 368.300

Lasso Coefficients Sorted by Importance:
            Feature  Lasso Coef  Lasso Importance
3            siconc  477.233899        477.233899
0                tp -472.419000        472.419000
18  day_of_year_cos -399.984381        399.984381
17  day_of_year_sin  -77.572118         77.572118
13     siconc_lag_1  -60.930261         60.930261
14     siconc_lag_2  -56.524503         56.524503
1               t2m   47.063523         47.063523
2               sst  -26.221294         26.221294
15     siconc_lag_3  -22.131921         22.131921
10        sst_lag_1  -18.254143         18.254143
11        sst_lag_2  -15.351382         15.351382
7         t2m_lag_1  -14.411427         14.411427
8         t2m_lag_2  -11.615316         11.615316
12        sst_lag_3  -11.503368         11.503368
16             year   -0.997732          0.997732
9         t2m_lag_3   -0.996228         

In [198]:
df = penguin_df
prep(df)
X, y, scaler, features, scaled_features = prep(df)
use_random_forest(X, y, features)

Random Forest Average R^2 Score: 0.522
Random Forest Average RMSE: 284.066
            Feature  Importance
2               sst    0.276126
18  day_of_year_cos    0.240799
3            siconc    0.065304
17  day_of_year_sin    0.047503
16             year    0.047169
1               t2m    0.043628
12        sst_lag_3    0.034521
10        sst_lag_1    0.031790
11        sst_lag_2    0.027775
7         t2m_lag_1    0.027616
5          tp_lag_2    0.024497
0                tp    0.023963
6          tp_lag_3    0.023302
8         t2m_lag_2    0.022342
4          tp_lag_1    0.016907
9         t2m_lag_3    0.016176
13     siconc_lag_1    0.011346
14     siconc_lag_2    0.010771
15     siconc_lag_3    0.008463


In [208]:
df = penguin_df
prep(df)
X, y, scaler, features, scaled_features = prep(df)
use_xgb(X, y)

XGBoost Average R^2 Score: 0.504
XGBoost Average RMSE: 286.420
            Feature  Importance
18  day_of_year_cos    0.413217
2               sst    0.119455
17  day_of_year_sin    0.068181
16             year    0.067367
3            siconc    0.045540
5          tp_lag_2    0.030590
7         t2m_lag_1    0.028579
12        sst_lag_3    0.028546
10        sst_lag_1    0.023605
8         t2m_lag_2    0.021775
1               t2m    0.021429
6          tp_lag_3    0.020392
9         t2m_lag_3    0.020153
11        sst_lag_2    0.019508
15     siconc_lag_3    0.018964
13     siconc_lag_1    0.016810
4          tp_lag_1    0.014222
14     siconc_lag_2    0.011655
0                tp    0.010012


# for species as a whole:


In [200]:
species = penguin_df['common_name'].unique()
for specie in species:
    print(specie)
    df = penguin_df[(penguin_df['common_name'] == specie)]
    prep(df)
    X, y, scaler, features, scaled_features = prep(df)
    use_ridge_lasso(X, y)

Adelie Penguin
Ridge Average R^2 Score: -1.347
Ridge Average RMSE: 227.959
Lasso Average R^2 Score: -1.326
Lasso Average RMSE: 227.121

Lasso Coefficients Sorted by Importance:
            Feature  Lasso Coef  Lasso Importance
3            siconc  288.878716        288.878716
17  day_of_year_sin  171.148487        171.148487
2               sst  -95.883206         95.883206
18  day_of_year_cos   91.475233         91.475233
15     siconc_lag_3  -62.830180         62.830180
12        sst_lag_3  -18.119210         18.119210
1               t2m  -14.293169         14.293169
7         t2m_lag_1    8.661194          8.661194
14     siconc_lag_2   -6.364093          6.364093
10        sst_lag_1   -6.242779          6.242779
11        sst_lag_2   -6.220027          6.220027
8         t2m_lag_2    5.704054          5.704054
13     siconc_lag_1   -5.301437          5.301437
16             year   -3.796648          3.796648
9         t2m_lag_3    1.858452          1.858452
6          tp_lag_3    

In [201]:
species = penguin_df['common_name'].unique()
for specie in species:
    print(specie)
    df = penguin_df[(penguin_df['common_name'] == specie)]
    prep(df)
    X, y, scaler, features, scaled_features = prep(df)
    use_random_forest(X, y, features)

Adelie Penguin
Random Forest Average R^2 Score: 0.214
Random Forest Average RMSE: 184.114
            Feature  Importance
2               sst    0.579680
16             year    0.149810
3            siconc    0.092810
18  day_of_year_cos    0.067687
17  day_of_year_sin    0.045445
1               t2m    0.010566
14     siconc_lag_2    0.007029
0                tp    0.005449
12        sst_lag_3    0.004410
15     siconc_lag_3    0.004388
11        sst_lag_2    0.004379
13     siconc_lag_1    0.004140
8         t2m_lag_2    0.004047
7         t2m_lag_1    0.003726
10        sst_lag_1    0.003662
9         t2m_lag_3    0.003573
4          tp_lag_1    0.003139
5          tp_lag_2    0.003055
6          tp_lag_3    0.003005
Chinstrap Penguin
Random Forest Average R^2 Score: 0.175
Random Forest Average RMSE: 437.258
            Feature  Importance
18  day_of_year_cos    0.459080
2               sst    0.078201
10        sst_lag_1    0.078039
17  day_of_year_sin    0.074733
1               t

In [ ]:
species = penguin_df['common_name'].unique()
for specie in species:
    print(specie)
    df = penguin_df[(penguin_df['common_name'] == specie)]
    prep(df)
    X, y, scaler, features, scaled_features = prep(df)
    use_xgb(X, y)

# Different colonies

In [ ]:
# loop through the sufficient pairs and apply the ridge and lasso regression
for pair in sufficient_pairs:
    print(pair['specie'])
    for colony in pair['colonies']:
        print(colony)
        df = penguin_df[(penguin_df['common_name'] == pair['specie']) & (penguin_df['colony_name'] == colony)]
        prep(df)
        X, y, scaler, features, scaled_features = prep(df)
        use_ridge_lasso(X, y)

In [ ]:
for pair in sufficient_pairs:
    print(pair['specie'])
    for colony in pair['colonies']:
        print(colony)
        df = penguin_df[(penguin_df['common_name'] == pair['specie']) & (penguin_df['colony_name'] == colony)]
        prep(df)
        X, y, scaler, features, scaled_features = prep(df)
        use_random_forest(X, y, features)

In [ ]:
for pair in sufficient_pairs:
    print(pair['specie'])
    for colony in pair['colonies']:
        print(colony)
        df = penguin_df[(penguin_df['common_name'] == pair['specie']) & (penguin_df['colony_name'] == colony)]
        prep(df)
        X, y, scaler, features, scaled_features = prep(df)
        use_xgb(X, y)